<a href="https://colab.research.google.com/github/charbells/1864093_ProcesamientoDeDatos/blob/main/1864093_Tarea5Codigo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importar librerias

import os
import pandas as pd
from PIL import Image
import numpy as np
import tensorflow as tf
import numpy as np
from PIL import Image
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

In [2]:
# Ruta a la carpeta principal donde están las carpetas de los animales
main_path = "/content/drive/MyDrive/ProcesamientoDatos/Animales"

# Crear una lista para almacenar datos
data = []

# Tomar imagenes de carpetas
for folder in os.listdir(main_path):
    folder_path = os.path.join(main_path, folder)
    if os.path.isdir(folder_path):
        for file in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file)
            if file.endswith(('.png', '.jpg', '.jpeg')):
                # Cargar imagen
                image = Image.open(file_path)
                # Convertir la imagen en array
                image_array = np.array(image)
                data.append({"image": image_array, "label": folder})

# DataFrame
df = pd.DataFrame(data)


In [3]:
# Ejemplo del DataFrame
print(df.head())

                                               image    label
0  [[[253, 253, 251], [253, 253, 251], [253, 253,...  Caballo
1  [[[247, 255, 250], [244, 255, 247], [240, 253,...  Caballo
2  [[[195, 222, 243], [193, 220, 241], [190, 217,...  Caballo
3  [[[50, 68, 42], [51, 69, 43], [60, 78, 54], [5...  Caballo
4  [[[158, 179, 196], [156, 177, 194], [153, 174,...  Caballo


In [4]:
# Todas las imagenes del mismo tamaño
IMG_SIZE = (64, 64)
resized_images = []

for img in df['image']:
    # Convertir a formato RGB
    resized_img = Image.fromarray(img).convert("RGB").resize(IMG_SIZE)
    resized_images.append(np.array(resized_img))

# Crear las variables X
X = np.array(resized_images)

# Crear variable y
y = np.array(df['label'].tolist())

# Normalizar las imágenes
X = X / 255.0


In [5]:
# Realizar One-hot encoding
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_categorical = to_categorical(y_encoded)  # One-hot encoding

# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.2, random_state=42)

# Construir el modelo secuencial
model = Sequential()

# Capa convolucional
model.add(Conv2D(25, kernel_size=(3, 3), strides=(1, 1), padding='valid', activation='relu', input_shape=X_train.shape[1:]))
model.add(MaxPool2D(pool_size=(1, 1)))
model.add(Flatten())

# Capa oculta
model.add(Dense(100, activation='relu'))

# Capa de salida
model.add(Dense(y_categorical.shape[1], activation='softmax'))

# Resumen del modelo
model.summary()

# Compilar el modelo
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

# Entrenar el modelo
train = model.fit(X_train, y_train, batch_size=128, epochs=10, validation_data=(X_test, y_test))


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 62, 62, 25)          │             700 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 62, 62, 25)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 96100)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 100)                 │       9,610,100 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 5)                   │             505 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 9,611,305 (36.66 MB)

 Trainable params: 9,611,305 (36.66 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 9s 58ms/step - accuracy: 0.3006 - loss: 3.9490 - val_accuracy: 0.4814 - val_loss: 1.2573
Epoch 2/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.5002 - loss: 1.2082 - val_accuracy: 0.5244 - val_loss: 1.1589
Epoch 3/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.5881 - loss: 1.0389 - val_accuracy: 0.5573 - val_loss: 1.1196
Epoch 4/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.6369 - loss: 0.9281 - val_accuracy: 0.5704 - val_loss: 1.0691
Epoch 5/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.6828 - loss: 0.8190 - val_accuracy: 0.5759 - val_loss: 1.0798
Epoch 6/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - accuracy: 0.7313 - loss: 0.7276 - val_accuracy: 0.5963 - val_loss: 1.0314
Epoch 7/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.7709 - loss: 0.6165 - val_accuracy: 0.6003 - val_loss: 1.0595
Epoch 8/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.8226 - loss: 0.5340 - val_accuracy: 0.5908 - v

In [8]:
# Evaluacion del modelo
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Accuracy en test: {test_acc}")


86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5781 - loss: 1.1241
Accuracy en test: 0.579868733882904


In [9]:
# Función para cargar y preprocesar imagen nueva
def preprocess_image(image_path, img_size=(64, 64)):
    # Cargar imagen
    img = Image.open(image_path)
    # Convertir a RGB
    img = img.convert("RGB")
    # Diemnsionar la imagen
    img = img.resize(img_size)
    # Convertir a array
    img_array = np.array(img) / 255.0  # Normalizar

    return np.expand_dims(img_array, axis=0)

# Ruta a la imagen a probar
test_image_path = "/content/drive/MyDrive/ProcesamientoDatos/Perro_prueba.jpg"

# Preprocesar la imagen
preprocessed_image = preprocess_image(test_image_path)

# Predicción
predictions = model.predict(preprocessed_image)

# Clase con mayor probabilidad
predicted_class_index = np.argmax(predictions)
predicted_label = label_encoder.inverse_transform([predicted_class_index])

print(f"Predicción: {predicted_label[0]}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Predicción: Perro
